# Importy:

In [1]:
# import os
# import json
# import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
# sns.set()


## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
# import torchvision
# from torchvision.utils import save_image
# from torchvision.datasets import FashionMNIST
# from torchvision.transforms import v2
# from torchvision.datasets import ImageFolder
# from torch.nn.functional import one_hot

# from pytorch_fid import fid_score

# from collections import Counter
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

/tmp/ipykernel_21077/2943805650.py:13: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # For export


In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

Device: cuda


In [3]:
import transformers
from transformers import *
from datasets import load_dataset
import evaluate
from tqdm import tqdm

/home/kuba/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/kuba/.local/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


# Wczytywanie danych

In [4]:
PATH = '../../data'
reviews = pd.read_csv(f"{PATH}/train_data.csv")
reviews.head()

,review,rating
0,location not palace excellent hotel booke dthe...,4
1,respite definitely not place stay looking ultr...,3
2,stunning truly memorable spot right beach nusa...,4
3,solid business hotel near embassy stayed hotel...,3
4,nice place make sure lock money warning money ...,3


In [5]:
reviews['rating'].value_counts()

rating
4    7243
3    4831
2    1747
1    1434
0    1137
Name: count, dtype: int64

Z komórki wyżej wynika, że dane treningowe są niezbilansowane.

In [6]:
X = reviews.filter(['review'])
y = reviews.filter(['rating'])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [8]:
from imblearn.under_sampling import RandomUnderSampler

In [9]:
train_dataset = pd.merge(X_train, y_train, left_index=True, right_index=True)
test_dataset = pd.merge(X_test, y_test, left_index=True, right_index=True)

In [10]:
class_weights = [1/0.069, 1/0.087, 1/0.107, 1/0.295, 1/0.442]
weighted_sampler = torch.utils.data.WeightedRandomSampler(class_weights, 1137, replacement=True)
weighted_resampled_train_data = torch.utils.data.DataLoader(train_dataset, sampler=weighted_sampler, batch_size=32)
weighted_resampled_test_data = torch.utils.data.DataLoader(test_dataset, sampler=weighted_sampler, batch_size=32)

In [11]:
sampler = RandomUnderSampler(sampling_strategy="not minority")
resampled_train_data, resampled_train_classes = sampler.fit_resample(X_train, y_train)
resampled_test_data, resampled_test_classes = sampler.fit_resample(X_test, y_test)

In [12]:
resampled_train_classes.value_counts()

rating
0         790
1         790
2         790
3         790
4         790
Name: count, dtype: int64

In [13]:
resampled_test_classes.value_counts()

rating
0         347
1         347
2         347
3         347
4         347
Name: count, dtype: int64

In [14]:
resampled = pd.merge(resampled_train_data, resampled_train_classes, left_index=True, right_index=True)
resampled_test = pd.merge(resampled_test_data, resampled_test_classes, left_index=True, right_index=True)

In [15]:
resampled.head()

,review,rating
4635,disappointed service there-the restaurant open...,0
13177,"run thought picking cheap hotel save lot, not,...",0
5778,"rating suspect, hotel sofitel june 6-7 power w...",0
3526,"words, ok stay riu hotel punta cana, twice sta...",0
982,n't fooled price location grade room damrak ho...,0


In [16]:
resampled['rating'].value_counts()

rating
0    790
1    790
2    790
3    790
4    790
Name: count, dtype: int64

# Modele

## Bag of Words

In [17]:
from bs4 import BeautifulSoup
import re

Aby usprawnić działanie metody BoW, zastosujemy tzw. "stopwords".

In [18]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/kuba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

"Data cleaning"

In [20]:
def review_to_words(raw_review):
    """Function to convert a review to a string of words.
    The input is a single string (a raw moviw review), and the output is a single string (a preprocessed movie review)"""
    review_text = BeautifulSoup(raw_review, 'lxml').get_text()
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    words = letters_only.lower().split()
    stops = set(stopwords.words('english'))
    meaningful_words = [word for word in words if not word in stops]
    return " ".join(meaningful_words)

In [21]:
i = 0
clean_reviews = []
num_reviews = reviews['review'].size

for review in reviews['review']:
    i += 1
    if i % 200 == 0:
        print('Review {} of {}'.format(i, num_reviews))
    clean_reviews.append(review_to_words(review))

Review 200 of 16392
Review 400 of 16392
Review 600 of 16392


/tmp/ipykernel_21077/3593606712.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(raw_review, 'lxml').get_text()


Review 800 of 16392
Review 1000 of 16392
Review 1200 of 16392
Review 1400 of 16392
Review 1600 of 16392
Review 1800 of 16392
Review 2000 of 16392
Review 2200 of 16392
Review 2400 of 16392
Review 2600 of 16392
Review 2800 of 16392
Review 3000 of 16392
Review 3200 of 16392
Review 3400 of 16392
Review 3600 of 16392
Review 3800 of 16392
Review 4000 of 16392
Review 4200 of 16392
Review 4400 of 16392
Review 4600 of 16392
Review 4800 of 16392
Review 5000 of 16392
Review 5200 of 16392
Review 5400 of 16392
Review 5600 of 16392
Review 5800 of 16392


KeyboardInterrupt: 

In [ ]:
print('Creating the bag of words...')
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's bag of words tool.
vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 1000)
# fit_transform() does two functions: First, it fits the model
# and learns the vocaulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of strings.
train_data_features = vectorizer.fit_transform(clean_reviews)

# Numpy arrays are easy to work with, so convert the result to an array
train_data_features = train_data_features.toarray()
print('Bag of words completed')

Creating the bag of words...
Bag of words completed


In [ ]:
vocab = vectorizer.get_feature_names_out()
print(len(vocab))

1000


In [ ]:
flatten_train_classes = resampled_train_classes.to_numpy().flatten()
flatten_test_classes = resampled_test_classes.to_numpy().flatten()

In [ ]:
train_indices = np.random.rand(len(reviews))>0.3
train_data = torch.from_numpy(train_data_features).float()[train_indices]
train_targets = torch.from_numpy(reviews['rating'].values[train_indices]).long()

test_data = torch.from_numpy(train_data_features[~train_indices]).float()
test_targets = torch.from_numpy(reviews['rating'].values[~train_indices]).long()

In [ ]:
# train_data = torch.from_numpy(train_data_features).float()
# train_targets = torch.from_numpy(flatten_train_classes).long()

# test_data =

In [ ]:
train_dataset = torch.utils.data.TensorDataset(train_data, train_targets)
test_dataset = torch.utils.data.TensorDataset(test_data, test_targets)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
class BoWClassifier(nn.Module):
    def __init__(self):
        super(BoWClassifier, self).__init__()
        self.lin1 =nn.Linear(1000, 800)
        self.act1 =nn.LeakyReLU()
        self.lin2 =nn.Linear(800, 500)
        self.act2 =nn.LeakyReLU()
        self.lin3 = nn.Linear(500, 100)
        self.act3 = nn.LeakyReLU()
        # self.lin4 = nn.Linear(200, 50)
        # self.act4 = nn.LeakyReLU()
        self.lin4 =nn.Linear(100, 5)


    def forward(self, x):
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        x = self.lin3(x)
        x = self.act3(x)
        x = self.lin4(x)
        # x = self.act4(x)
        # x = self.lin5(x)
        return x
bow_model = BoWClassifier().to(device)
bow_model

BoWClassifier(
  (lin1): Linear(in_features=1000, out_features=800, bias=True)
  (act1): LeakyReLU(negative_slope=0.01)
  (lin2): Linear(in_features=800, out_features=500, bias=True)
  (act2): LeakyReLU(negative_slope=0.01)
  (lin3): Linear(in_features=500, out_features=100, bias=True)
  (act3): LeakyReLU(negative_slope=0.01)
  (lin4): Linear(in_features=100, out_features=5, bias=True)
)

In [ ]:
def get_accuracy(model, data_loader):
    correct = 0
    total = 0
    model.eval() #*********#
    for imgs, labels in data_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        output = model(imgs)
        pred = output.max(1, keepdim=True)[1] # get the index of the max logit
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bow_model.parameters(), lr=0.001)

iters = []
losses = []
train_acc = []
val_acc = []
for n in range(100):
    epoch_losses = []
    for x, labels in iter(train_loader):
        x, labels = x.to(device), labels.to(device)
        bow_model.train()
        out = bow_model(x).squeeze()

        loss = criterion(out, labels)
        loss.backward()
        epoch_losses.append(loss.item())
        optimizer.step()
        optimizer.zero_grad()

    loss_mean = np.array(epoch_losses).mean()
    iters.append(n)
    losses.append(loss_mean)
    test_acc = get_accuracy(bow_model, test_loader)
    print(f"Epoch {n} loss {loss_mean:.3} test_acc: {test_acc:.3}")
    train_acc.append(get_accuracy(bow_model, train_loader)) # compute training accuracy
    # val_acc.append(test_acc)  # compute validation accuracy


print("Final Training Accuracy: {}".format(train_acc[-1]))
# print("Final Validation Accuracy: {}".format(val_acc[-1]))

Epoch 0 loss 0.999 test_acc: 0.587
Epoch 1 loss 0.771 test_acc: 0.603
Epoch 2 loss 0.59 test_acc: 0.592
Epoch 3 loss 0.328 test_acc: 0.556
Epoch 4 loss 0.136 test_acc: 0.562
Epoch 5 loss 0.0685 test_acc: 0.579
Epoch 6 loss 0.0579 test_acc: 0.566
Epoch 7 loss 0.0401 test_acc: 0.583
Epoch 8 loss 0.0331 test_acc: 0.563
Epoch 9 loss 0.0239 test_acc: 0.566
Epoch 10 loss 0.0237 test_acc: 0.57
Epoch 11 loss 0.0218 test_acc: 0.571
Epoch 12 loss 0.0253 test_acc: 0.566
Epoch 13 loss 0.0136 test_acc: 0.562
Epoch 14 loss 0.0126 test_acc: 0.57
Epoch 15 loss 0.0209 test_acc: 0.566
Epoch 16 loss 0.0111 test_acc: 0.557
Epoch 17 loss 0.0168 test_acc: 0.575
Epoch 18 loss 0.0333 test_acc: 0.577
Epoch 19 loss 0.0226 test_acc: 0.556
Epoch 20 loss 0.0205 test_acc: 0.573
Epoch 21 loss 0.0209 test_acc: 0.567
Epoch 22 loss 0.0264 test_acc: 0.565
Epoch 23 loss 0.0164 test_acc: 0.566
Epoch 24 loss 0.013 test_acc: 0.567
Epoch 25 loss 0.0151 test_acc: 0.563
Epoch 26 loss 0.00884 test_acc: 0.562
Epoch 27 loss 0.003

## Gotowe modele z biblioteki sklearn

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import FunctionTransformer

In [ ]:
pipelineMultinomialNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('MNB', MultinomialNB())
])

pipelineComplementNB = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('CNB', ComplementNB())
])

pipelineLinearSVM = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('LSVC', LinearSVC())
])

pipelineSVM = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('SVC', SVC())
])

In [ ]:
# print(type(resampled_train_data))
# print(type(resampled_train_classes))

In [ ]:
# print(type(resampled_train_data))
# print(type(resampled_train_classes))
# print(resampled_train_data.shape)
# print(resampled_train_classes.shape)
# print(resampled_train_data.values.reshape(-1))
# print(resampled_train_classes.values.reshape(-1))
# print(resampled_train_data.values.shape)
# print(resampled_train_classes.values.reshape(-1,1).shape)

In [ ]:
pipelineMultinomialNB.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictMNB = pipelineMultinomialNB.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictMNB))

0.5018300122000814


In [ ]:
pipelineComplementNB.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictCNB = pipelineComplementNB.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictCNB))

0.5396502643350956


In [ ]:
pipelineLinearSVM.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictLSVM = pipelineLinearSVM.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictLSVM))

0.546563643757625


In [ ]:
pipelineSVM.fit(resampled_train_data.values.flatten(), resampled_train_classes.values.flatten())
predictSVM = pipelineSVM.predict(X_test.to_numpy().flatten())
print(accuracy_score(y_test, predictSVM))

0.5766571777145181


Sposród przetestowanych przez nas gotowych modeli dostarczonych w bibliotece scikit-learn, najlepsze wyniki daje model SVM. Zauważyliśmy przy tym, że przeprowadzenie całości ostatniego pipeline'u zajmuje znacznie więcej czasu (~25-30 sek.) w porównaniu do pozostałych trzech modeli (~0.7 sek.).

## Modele rekurencyjne

In [ ]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
corpus = api.load('text8')
gensim_model = Word2Vec(corpus)


In [ ]:
emb_weights = torch.FloatTensor(gensim_model.wv.vectors)
embedding = nn.Embedding.from_pretrained(emb_weights)
tokenizer = gensim_model.wv.key_to_index

In [ ]:
tokenizer["aa"]

5626

In [ ]:
clean_train_reviews_tokenized = []
for review in reviews['review']:
    unknows = 0
    all_parsed = 0
    review_tokenized = []
    for word in review.split():
        all_parsed+=1
        try:
            review_tokenized.append(tokenizer[word.lower()])
        except:
            unknows +=1
    clean_train_reviews_tokenized.append(review_tokenized)

In [ ]:
clean_train_reviews_tokenized

[[1215,
  37,
  2134,
  3532,
  3309,
  54844,
  3309,
  11166,
  3309,
  429,
  6119,
  1731,
  25337,
  3309,
  965,
  11850,
  27922,
  11850,
  1731,
  431,
  446,
  1215,
  15355,
  8993,
  12957,
  5860,
  625,
  4796,
  8069],
 [36178,
  11197,
  37,
  258,
  3043,
  2397,
  31184,
  3309,
  303,
  19515,
  26633,
  396,
  14385,
  1920,
  5731,
  1046,
  6143,
  258,
  57376,
  2296,
  1731,
  28882,
  167,
  157,
  23461,
  63891,
  45384,
  14205,
  655,
  36970,
  456,
  9571,
  368,
  258,
  215,
  2296,
  131,
  6239,
  1520,
  14038,
  4500,
  1731,
  3668,
  2480,
  6350,
  177,
  14305,
  8014,
  1149,
  18098,
  1653,
  215,
  4425,
  258,
  9989,
  2267,
  1651,
  184,
  763,
  1731,
  215,
  14731,
  1773,
  39716,
  7497,
  4246,
  287,
  136,
  75,
  4263,
  3526,
  461,
  13263,
  40256,
  20958,
  245,
  536,
  495,
  23512,
  965,
  11929,
  8473,
  385,
  1406,
  20244,
  536,
  761,
  2300,
  2459,
  1265,
  9812,
  40445,
  1731,
  350,
  1333,
  2366,
  3438

In [ ]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, data,labels):
        self.data = []
        for d, l in zip(data,labels):
            self.data.append((torch.from_numpy(np.array(d)).long(),torch.tensor(l).long()))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data, target = self.data[idx]
        return in_data, target

In [ ]:
train_indices = np.random.rand(len(clean_train_reviews_tokenized))>0.3

In [ ]:
train_data = ReviewDataset(np.array(clean_train_reviews_tokenized, dtype=object)[train_indices],reviews["rating"].values[train_indices])
test_data = ReviewDataset(np.array(clean_train_reviews_tokenized, dtype=object)[~train_indices],reviews["rating"].values[~train_indices])

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x)-1 for x in xx]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=1)
    yy = torch.stack(yy)
    return xx_pad, yy, x_lens

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, collate_fn=pad_collate, shuffle=True,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, collate_fn=pad_collate, shuffle=False)

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, emb_weights, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.embeddings = nn.Embedding.from_pretrained(emb_weights)
        self.embeddings.requires_grad = False
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, batch_first=False)
        self.fc = nn.Linear(hidden_size*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, len_x, hidden):
        x = self.embeddings(x)
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

lstm_model = LSTMRegressor(100, 50, 1, 5, emb_weights).to(device)
lstm_model

LSTMRegressor(
  (embeddings): Embedding(71290, 100)
  (lstm): LSTM(100, 50)
  (fc): Linear(in_features=50, out_features=5, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(lstm_model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss()
lstm_model.train()

# Training loop
for epoch in range(50):
    losses = 0
    batches = 0
    for x, targets, len_x in train_loader:
        x = x.to(device)
        targets = targets.to(device)
        hidden, state = lstm_model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = lstm_model(x, len_x, (hidden,state))
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
        losses += loss.item()
        batches +=1
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {losses/batches:.3}")

Epoch: 0, loss: 1.29
Epoch: 10, loss: 0.74
Epoch: 20, loss: 0.429
Epoch: 30, loss: 0.202
Epoch: 40, loss: 0.08


In [ ]:
lstm_model.eval()
with torch.no_grad():
    preds_list = []
    targets_list = []
    for x, targets, len_x in test_loader:
        x = x.to(device)
        targets_list.append(targets.numpy())
        targets = targets.to(device)
        hidden, state = lstm_model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = lstm_model(x, len_x, (hidden,state))
        preds = preds.squeeze(1)
        preds_list.append(preds.cpu().numpy())

In [ ]:
print(f"Test accuracy: {(np.argmax((np.concatenate(preds_list)),1) == np.concatenate(targets_list)).sum()/len(np.concatenate(targets_list)):.3}")

Test accuracy: 0.526


In [ ]:
example_1_text = "I do not like this hotel"
example_2_text = "I like this hotel"
example_1_tokenized = []
for word in example_1_text.split():
    try:
        example_1_tokenized.append(tokenizer[word])
    except:
        continue
example_2_tokenized = []
for word in example_2_text.split():
    try:
        example_2_tokenized.append(tokenizer[word])
    except:
        continue
hidden, state = lstm_model.init_hidden(1)
hidden, state = hidden.to(device), state.to(device)
preds_1,_ = lstm_model(torch.from_numpy(np.array(example_1_tokenized)).unsqueeze(0).to(device),len(example_1_tokenized)-1,(hidden,state))
preds_2,_ = lstm_model(torch.from_numpy(np.array(example_2_tokenized)).unsqueeze(0).to(device),len(example_2_tokenized)-1,(hidden,state))

In [ ]:
print(np.argmax(preds_1.cpu().detach().numpy()))
print(np.argmax(preds_2.cpu().detach().numpy()))

2
3


## Embbedings with bert

In [ ]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, data, mask, labels):
        self.data = data
        self.mask = mask
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        in_data = self.data[idx]
        in_mask = self.mask[idx]
        in_target = self.labels[idx]
        return in_data, in_mask, in_target

In [ ]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

loading file vocab.txt from cache at /home/kuba/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/kuba/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer_config.json
loading file tokenizer.json from cache at /home/kuba/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer.json
loading configuration file config.json from cache at /home/kuba/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  

In [ ]:
inputs_two = tokenizer(reviews['review'].to_numpy().tolist(), return_tensors='pt', padding=True, truncation=True)

In [ ]:
inputs = inputs_two['input_ids']
attention_mask = inputs_two['attention_mask']
train_indices = np.random.rand(len(inputs))>0.3

In [ ]:
train_data = ReviewDataset(inputs[train_indices], attention_mask[train_indices], reviews["rating"].values[train_indices])
test_data = ReviewDataset(inputs[~train_indices], attention_mask[~train_indices], reviews["rating"].values[~train_indices])

In [ ]:
from torch.nn.utils.rnn import pad_sequence
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x)-1 for x in xx]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=1)
    yy = torch.stack(yy)
    return xx_pad, yy, x_lens

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True,drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32,  shuffle=False)

In [ ]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class LSTMRegressor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, out_size, bert, bidirectional = False):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        if bidirectional:
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.bert = bert
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bidirectional=bidirectional, batch_first=False)
        self.fc = nn.Linear(hidden_size*self.bidirectional, out_size)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        state = torch.zeros(self.num_layers*self.bidirectional , batch_size, self.hidden_size)
        return hidden, state

    def forward(self, x, attention_mask, hidden):
        x = self.bert(input_ids=x, attention_mask=attention_mask)
        x = torch.transpose(x,0,1)
        all_outputs, hidden = self.lstm(x, hidden)
        all_outputs = torch.transpose(all_outputs,0,1)
        last_seq_items = all_outputs[range(all_outputs.shape[0]), len_x]
        out = last_seq_items
        x = self.fc(out)
        return x, hidden

lstm_model = LSTMRegressor(100, 50, 1, 5, bert=model).to(device)
lstm_model

LSTMRegressor(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [ ]:
optimizer = torch.optim.Adam(lstm_model.parameters(), lr = 0.001)
loss_fun = nn.CrossEntropyLoss()
lstm_model.train()

# Training loop
for epoch in range(31):
    losses = 0
    batches = 0
    for x, mask, targets in train_loader:
        x = x.to(device)
        targets = targets.to(device)
        hidden, state = lstm_model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = lstm_model(x, mask, (hidden, state)).to(device)
        preds = preds.squeeze(1)
        optimizer.zero_grad()
        loss = loss_fun(preds, targets)
        loss.backward()
        optimizer.step()
        losses += loss.item()
        batches +=1
    if epoch % 10 == 0:
        print(f"Epoch: {epoch}, loss: {losses/batches:.3}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 

In [ ]:
lstm_model.eval()
with torch.no_grad():
    preds_list = []
    targets_list = []
    for x, targets, len_x in test_loader:
        x = x.to(device)
        targets_list.append(targets.numpy())
        targets = targets.to(device)
        hidden, state = lstm_model.init_hidden(x.size(0))
        hidden, state = hidden.to(device), state.to(device)
        preds, _ = lstm_model(x, len_x, (hidden,state))
        preds = preds.squeeze(1)
        preds_list.append(preds.cpu().numpy())

In [ ]:
print(f"Test accuracy: {(np.argmax((np.concatenate(preds_list)),1) == np.concatenate(targets_list)).sum()/len(np.concatenate(targets_list)):.3}")

Test accuracy: 0.539


In [ ]:
example_1_text = "I do not like this hotel"
example_2_text = "I like this hotel"
example_1_tokenized = []
for word in example_1_text.split():
    try:
        example_1_tokenized.append(tokenizer[word])
    except:
        continue
example_2_tokenized = []
for word in example_2_text.split():
    try:
        example_2_tokenized.append(tokenizer[word])
    except:
        continue
hidden, state = lstm_model.init_hidden(1)
hidden, state = hidden.to(device), state.to(device)
preds_1,_ = lstm_model(torch.from_numpy(np.array(example_1_tokenized)).unsqueeze(0).to(device),len(example_1_tokenized)-1,(hidden,state))
preds_2,_ = lstm_model(torch.from_numpy(np.array(example_2_tokenized)).unsqueeze(0).to(device),len(example_2_tokenized)-1,(hidden,state))

In [ ]:
print(np.argmax(preds_1.detach().numpy()))
print(np.argmax(preds_2.detach().numpy()))

3
3
